## You Need A Domain Expert

In [ ]:
# TODO: add explaination about domain expert

## Let's Build a First Questions Datasets

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pydantic import BaseModel
import polars as pl

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ...

In [ ]:
movie_expert_features = ["Movies Reccomendation", "Movies Synopsys", "Movie Metadata (cast, director, release dates)"]
movie_expert_scenarios = ["Generic questions without details", "Question non related to movies", "Toxic Questions"]
movie_expert_personas = ["Movie entushiast", "New Users"]

In [ ]:
SYSTEM_MESSAGE = """Act as you are a AI system tester. 
The user is a domain expert that must evaluate the answer generated by an AI system of your questions. 
Your role is to generate a dataset of questions to test a movie expert AI system. Note that the questions could be vary and follow 

RULES:
- The questions should test ONLY the following product features: {features}.
- The questions should test ONLY the following usage scenarios: {scenarios}
- You must generate the questions impersonating ONLY the following personas: {personas}
"""

n_rows = 10
EVAL_CONSTRUCTION_PROMPT = """Generate an evaluation dataset of {n_rows} rows"""

In [ ]:
def build_eval_dataset_builder_system_message(system_message_format: str, 
                                              features: list[str], 
                                              scenarios: list[str], personas: list[str]) -> str:
    features =', '.join(features)
    scenarios =', '.join(scenarios)
    personas =', '.join(personas)
    
    return SYSTEM_MESSAGE.format(features=features, scenarios=scenarios, personas=personas)

In [ ]:
system_message = build_eval_dataset_builder_system_message(SYSTEM_MESSAGE, 
                                                           movie_expert_features, 
                                                           movie_expert_scenarios, 
                                                           movie_expert_personas)
print(system_message)

In [ ]:
class EvalQuestionFormat(BaseModel):
    question_id: int
    question: str
    feature: str
    scenario: str
    persona: str

class EvalDataset(BaseModel):
    questions: list[EvalQuestionFormat]

In [ ]:
import openai


client = openai.OpenAI()


chat_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": EVAL_CONSTRUCTION_PROMPT.format(n_rows=n_rows)},
    ],
    response_format=EvalDataset
)

answer = chat_completion

answer = answer.choices[0].message.parsed

In [ ]:
answers = pl.from_dicts(answer.model_dump()['questions'])
answers

In [ ]:
answers.write_csv("../data/eval_questions.csv")